In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

In [3]:
from pprint import pprint
from collections import defaultdict

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
        for document in documents]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [4]:
from gensim import corpora

dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict')
print(dictionary)

2021-03-22 11:43:10,964 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-03-22 11:43:10,964 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2021-03-22 11:43:10,965 : INFO : saving Dictionary object under /tmp/deerwester.dict, separately None
2021-03-22 11:43:10,966 : INFO : saved /tmp/deerwester.dict


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [5]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [6]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
new_vec

[(0, 1), (1, 1)]

In [7]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)
print(corpus)

2021-03-22 11:43:15,026 : INFO : storing corpus in Matrix Market format to /tmp/deerwester.mm
2021-03-22 11:43:15,028 : INFO : saving sparse matrix to /tmp/deerwester.mm
2021-03-22 11:43:15,029 : INFO : PROGRESS: saving document #0
2021-03-22 11:43:15,030 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2021-03-22 11:43:15,032 : INFO : saving MmCorpus index to /tmp/deerwester.mm.index


[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


In [9]:
from smart_open import open  # for transparently opening remote files


class MyCorpus:
    def __iter__(self):
        for line in open('tuto_db.txt'):
            # assume there is only one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

In [10]:
corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory
print(corpus_memory_friendly)

for vector in corpus_memory_friendly:
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


In [13]:
# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in open('tuto_db.txt'))
# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword]
           for stopword in stoplist
           if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)
dictionary.compactify()
print(dictionary)

2021-03-22 12:49:46,071 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-03-22 12:49:46,073 : INFO : built Dictionary(42 unique tokens: ['abc', 'applications', 'computer', 'for', 'human']...) from 9 documents (total 69 corpus positions)


to
Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [17]:
## Corpus Formats

# save corpus
corpus = [[(1, 0.5)], []]
corpora.MmCorpus.serialize('/tmp/corpus.mm', corpus)

# load corpus
corpus = corpora.MmCorpus('/tmp/corpus.mm')
print(corpus)
print(list(corpus))

2021-03-22 12:54:38,946 : INFO : storing corpus in Matrix Market format to /tmp/corpus.mm
2021-03-22 12:54:38,949 : INFO : saving sparse matrix to /tmp/corpus.mm
2021-03-22 12:54:38,950 : INFO : PROGRESS: saving document #0
2021-03-22 12:54:38,951 : INFO : saved 2x2 matrix, density=25.000% (1/4)
2021-03-22 12:54:38,953 : INFO : saving MmCorpus index to /tmp/corpus.mm.index
2021-03-22 12:54:38,955 : INFO : loaded corpus index from /tmp/corpus.mm.index
2021-03-22 12:54:38,956 : INFO : initializing cython corpus reader from /tmp/corpus.mm
2021-03-22 12:54:38,958 : INFO : accepted corpus with 2 documents, 2 features, 1 non-zero entries


MmCorpus(2 documents, 2 features, 1 non-zero entries)
[[(1, 0.5)], []]


In [23]:
import gensim
import numpy as np

numpy_matrix = np.random.randint(10, size=[2, 5])
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)

print(numpy_matrix)
print(list(corpus)) # jpp ca lit les vecteurs en colonne --'

[[4 1 2 7 8]
 [6 4 1 4 2]]
[[(0, 4.0), (1, 6.0)], [(0, 1.0), (1, 4.0)], [(0, 2.0), (1, 1.0)], [(0, 7.0), (1, 4.0)], [(0, 8.0), (1, 2.0)]]


In [28]:
import scipy.sparse

scipy_sparse_matrix = scipy.sparse.random(5,2)
corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
scipy_csc_matrix = gensim.matutils.corpus2csc(corpus)


print(scipy_csc_matrix)
#print(corpus)